## Examples - using yy_fmri_kit for intersubject correlation (ISC)

This notebook contains examples that demonstrate how to use the yy_fmri_kit package to compute and visualize intersubject correlation (ISC) in fMRI data.

Step 0: project specific - load behavioral data to define groups

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/path/to/your/file.csv")
df.columns

In [ ]:
# Drop rows with missing bids_id data
df = df.dropna(subset=["bids_id"])

# --- Example grouping --- 
# Map participants (bids_id) to political groups
group_map = dict(zip(df["bids_id"], df["political_group"]))
print(group_map)

In [ ]:
# --- Example grouping --- 
# Separate subjects (runs roots) into left and right groups
left_subjects  = [sub for sub, grp in group_map.items() if grp == "left"]
right_subjects = [sub for sub, grp in group_map.items() if grp == "right"]

print("Left group:", left_subjects)
print("Right group:", right_subjects)

Step 1: configure and run ISC analysis

In [ ]:
# compute parcelwise ISC for a given task across multiple subjects

from pathlib import Path
import pandas as pd
import importlib

from yy_fmri_kit.isc import timeseries
importlib.reload(timeseries)
from yy_fmri_kit.static.isc import config
importlib.reload(config)

from yy_fmri_kit.isc.parcel import (
    run_parcelwise_isc)
from yy_fmri_kit.postproc.parcellation import _resolve_atlas_and_labels

# --- Config ---
config = config.ISCConfig(
    derivatives_dir = Path("/path/to/fmri_analysis/data/derivatives/parcellated_ts"),
    output_dir      = Path("/path/to/fmri_analysis/data/derivatives/isc_ts"),
    subjects        = ["sub-1", "sub-2", "sub-4", "sub-5", "sub-6"], # or group lists, i.e. "left_subjects" and "right_subjects"--- # TODO: define a helper function to run isc on groups
    tf_template     = "MNI152NLin2009cAsym",
    tf_atlas        = "Schaefer2018",
    tf_desc         = "400Parcels7Networks",
    tf_resolution   = 2,
)

# 1) Load parcel time series
parcel_data = timeseries.load_all_subject_parcel_data(config, tasks=["AntiLeft"])

# 2) Get labels via your parcellation resolver (directly)
atlas_nii, labels_path = _resolve_atlas_and_labels(
    atlas_nii          = config.atlas_nii,
    labels_file        = config.labels_file,
    tf_template        = config.tf_template,
    tf_atlas           = config.tf_atlas,
    tf_desc            = config.tf_desc,
    tf_resolution      = config.tf_resolution,
    suffix             = "dseg",
)
labels_df = pd.read_csv(labels_path, sep="\t")

# 3) Parcelwise ISC
isc_df = run_parcelwise_isc(
    subject_data  = parcel_data,
    config        = config,
    task          = "AntiLeft",
    parcel_labels = labels_df,
)
save_path = config.output_dir / "parcelwise_isc_AntiLeft.tsv"
isc_df.to_csv(save_path, sep="\t", index=False)
print(f"Parcelwise ISC saved to: {save_path}")
print("Parcelwise ISC shape:", isc_df.shape)
print(isc_df.head())


In [ ]:
# visualize the ISC results on the brain
import numpy as np
import nibabel as nib
from nilearn import plotting


atlas_img = nib.load(atlas_nii)         # <--- this fixes the AttributeError
atlas_data = atlas_img.get_fdata().astype(int)

# ISC vector: length P (here P=400 for Schaefer-400)
isc_df = pd.read_csv(save_path, sep="\t")
isc_vals = isc_df["ISC"].to_numpy()
P = len(isc_vals)

# Assume Schaefer IDs are 1..P in the same order as labels_df / isc_df
label_ids = np.arange(1, P + 1)

isc_vol = np.zeros_like(atlas_data, dtype=float)
for lab, val in zip(label_ids, isc_vals):
    isc_vol[atlas_data == lab] = val
task = "AntiLeft"
isc_img = nib.Nifti1Image(isc_vol, atlas_img.affine, atlas_img.header)
isc_nifti_path = config.output_dir / f"parcelwise_isc_{task}_space-MNI152_Schaefer400.nii.gz"
isc_img.to_filename(isc_nifti_path)
print("ISC NIfTI saved to:", isc_nifti_path)

# Nice threshold: e.g., 75th percentile
thr = np.nanpercentile(isc_vals, 75)

plotting.plot_stat_map(
    isc_img,
    threshold=thr,
    display_mode="mosaic",   # or 'x', 'y', 'z'
    cut_coords=10,
    title="Parcelwise ISC – AntiLeft (Schaefer-400)"
)

In [ ]:
# TODO: debugg visualization function

In [ ]:
import importlib
from yy_fmri_kit.visualization import visualization
importlib.reload(visualization)
from yy_fmri_kit.visualization.visualization import plot_stat_niimg

task = "AntiLeft"
isc_img_path = "/path/to/fmri_analysis/data/derivatives/isc_ts/" + f"parcelwise_isc_{task}_space-MNI152_Schaefer400.nii.gz"

plot_stat_niimg(
    isc_img_path,
    title=f"Parcelwise ISC – {task} (Schaefer-400)",
    percentile=75,
)


In [ ]:
# compare isc before and after time shifting for QC

import pandas as pd

pre_path = "/path/to/fmri_analysis/data/derivatives/isc/parcelwise_isc.tsv"       # before time-shift
post_path = "/path/to/fmri_analysis/data/derivatives/isc_ts/parcelwise_isc.tsv"   # after time-shift

pre = pd.read_csv(pre_path, sep="\t")
post = pd.read_csv(post_path, sep="\t")

key_cols = ["name"]   # or ["parcel"], or ["index","name"] depending on your TSV
df = pre.merge(post, on=key_cols, suffixes=("_pre", "_ts"))
print(pre.columns)
print(post.columns)

import numpy as np

df["delta_ISC"] = df["ISC_ts"] - df["ISC_pre"]
df["pct_change"] = df["delta_ISC"] / df["ISC_pre"].replace(0, np.nan) * 100

print("Mean ISC pre:", df["ISC_pre"].mean())
print("Mean ISC ts:", df["ISC_ts"].mean())
print("Mean delta:", df["delta_ISC"].mean())
print("Median delta:", df["delta_ISC"].median())
print("Parcels with ISC increase:", (df["delta_ISC"] > 0).mean())

df_top = df.sort_values("delta_ISC", ascending=False)
print(df_top[["name", "ISC_pre", "ISC_ts", "delta_ISC"]].head(20))

df_worse = df.sort_values("delta_ISC", ascending=True)
print(df_worse[["name", "ISC_pre", "ISC_ts", "delta_ISC"]].head(20))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,5))
sns.histplot(df["delta_ISC"], bins=40, kde=True)

plt.axvline(0, color="black", linestyle="--", linewidth=1)

plt.title("ΔISC Distribution (Time-shifted − Original)")
plt.xlabel("ΔISC")
plt.ylabel("Parcel count")

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

plt.figure(figsize=(8,5))

inc = df["delta_ISC"][df["delta_ISC"] > 0]
dec = df["delta_ISC"][df["delta_ISC"] <= 0]

plt.hist(inc, bins=40, alpha=0.7, label="Increased", color="green")
plt.hist(dec, bins=40, alpha=0.7, label="Decreased", color="red")

plt.axvline(0, color="black", linestyle="--")
plt.legend()
plt.title("ΔISC: Increases vs Decreases")
plt.xlabel("ΔISC")
plt.ylabel("Parcel count")

plt.tight_layout()
plt.show()


In [ ]:
# TODO: add voxelwise ISC for all runs, autosave ISC results as both NIfTI and TSV

In [ ]:
# TODO: add visualzation functions for ISC results and modify later to inhance visuals

In [ ]:
# ROI voxelwise ISC computation example
import importlib
from pathlib import Path
import numpy as np
from yy_fmri_kit.isc import roi
importlib.reload(roi)
from yy_fmri_kit.isc.roi import compute_roi_isc_for_run

DERIV     = Path("/path/to/fmri_analysis/data/derivatives/denoised")
TIMESHIFT = Path("/path/to/fmri_analysis/data/derivatives/denoised_ts")
ROI       = Path("/path/to/fmri_analysis/data/derivatives/roi_masks/auditory_sphere_roi.nii.gz")

isc_img_pre, isc_vec_pre, _ = compute_roi_isc_for_run(
    derivatives_dir=DERIV,
    roi_mask_img=ROI,
    run_idx=2,
)

isc_img_post, isc_vec_post, _ = compute_roi_isc_for_run(
    derivatives_dir=TIMESHIFT,
    roi_mask_img=ROI,
    run_idx=2,
)
delta = isc_vec_post - isc_vec_pre
print("Mean ΔISC:", delta.mean())
print("Median ΔISC:", np.median(delta))
print("ΔISC > 0:", (delta > 0).mean())

import matplotlib.pyplot as plt
plt.hist(delta, bins=40)
plt.axvline(0, ls="--", color="black")
plt.show()



In [ ]:
import numpy as np
import nibabel as nib

# isc_img_pre and isc_img_post are Nifti1Image
pre_data  = isc_img_pre.get_fdata()
post_data = isc_img_post.get_fdata()

delta_data = post_data - pre_data  # 3D numpy array

delta_img = nib.Nifti1Image(delta_data, isc_img_pre.affine, isc_img_pre.header)


from nilearn import plotting
plotting.plot_stat_map(
    delta_img,
    title="ΔISC (post - pre)",
    cmap="cold_hot",
    threshold=0.0,
)